In [65]:
import os
from glob import glob
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm 

In [66]:
risultati = [y for x in os.walk("final_joined_result/") for y in glob(os.path.join(x[0], '*.csv'))]
risultati[:5]

['final_joined_result/triangle2_y.zip ExKMC.csv',
 'final_joined_result/churnONEHOT_y.zip-12_k skl_bis_kmeans.csv',
 'final_joined_result/compas-scores-two-yearsONEHOT_y.zip-2_True_k kmeans+dt.csv',
 'final_joined_result/s-set1_y.zip-4_True_k kmeans+dt.csv',
 'final_joined_result/wine.zip VarianceParTree.csv']

In [67]:
risultati_groupby_alg = dict()
risultati_groupby_dataset = dict()

for filename in risultati:
    alg = filename.split(" ")[-1][:-4]
    dataset = filename.split(" ")[0].split("\\")[-1][:-4]
    
    if alg not in risultati_groupby_alg:
        risultati_groupby_alg[alg] = []
    if dataset not in risultati_groupby_dataset:
        risultati_groupby_dataset[dataset] = []
    
    risultati_groupby_alg[alg].append(filename)
    risultati_groupby_dataset[dataset].append(filename)

In [68]:
res_values_uns = ["time", "silhouette", "calinski_harabasz", "davies_bouldin"]
res_values_sup = ["r_score", "adj_rand", "mut_info_score", "adj_mutual_info_score", "norm_mutual_info_score",
                                 "homog_score", "complete_score", "v_msr_score", "fwlks_mallows_score"]
res_values_all = res_values_uns + res_values_sup

In [69]:
df = pd.DataFrame()
lista = []

datasets = set()
algorithms = set()

for filename in risultati:
    alg = filename.split(" ")[-1][:-4]
    dataset = filename.split(" ")[0].split("/")[-1][:-4].replace(".zip", "").replace("-", "")
        
    
    datasets.add(dataset)
    algorithms.add(alg)
    
    lista.append(pd.read_csv(filename))
    
    lista[-1]["alg"] = alg
    if "wine" in dataset:
        lista[-1]["dataset"] = "wine"
    elif "home" in dataset:
        lista[-1]["dataset"] = "home"
    elif "ficoONEHOT" in dataset:
        lista[-1]["dataset"] = "ficoONEHOT"
    elif "fico" in dataset:
        lista[-1]["dataset"] = "fico"
    elif "bank" in dataset:
        lista[-1]["dataset"] = "bank"
    else:
        lista[-1]["dataset"] = dataset.split("_y")[0]+"_y"
    
df = pd.concat(lista)

df.columns

Index(['n_clusters', 'max_leaves', 'base_tree', 'max_nbr_values',
       'max_nbr_values_cat', 'time', 'silhouette', 'calinski_harabasz',
       'davies_bouldin', 'r_score', 'adj_rand', 'mut_info_score',
       'adj_mutual_info_score', 'norm_mutual_info_score', 'homog_score',
       'complete_score', 'v_msr_score', 'fwlks_mallows_score', 'alg',
       'dataset', 'init', 'n_init', 'max_iter', 'tol', 'verbose',
       'random_state', 'copy', 'algorithm', 'bisecting_strategy',
       'labels_as_tree_leaves', 'criterion', 'splitter', 'min_samples_split',
       'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features',
       'max_leaf_nodes', 'min_impurity_decrease', 'class_weight', 'ccp_alpha',
       'max_depth', 'max_nbr_clusters', 'bic_eps', 'eps', 'min_samples',
       'metric', 'n_jobs', 'linkage', 'amount_initial_centers', 'kmax',
       'criteria_clf', 'criteria_reg', 'depth_factor', 'metric_con',
       'metric_cat', 'max_eps', 'n_components', 'oblique_splits',
       'max_o

In [70]:
df.dataset.unique()

array(['triangle2_y', 'churnONEHOT_y', 'compasscorestwoyearsONEHOT_y',
       'sset1_y', 'wine', 'churn_y', '2d10c_y', 'compasscorestwoyears_y',
       'diabetes_y', 'longsquare_y', 'iris_y', 'sset2_y', 'tetra_y',
       'aggregation_y', 'wdbc_y', 'adultONEHOT_y', 'zelnik5_y', '2dd31_y',
       '2d3cno123_y', 'curet24k_y', '2d4c_y', 'ionosphere_y',
       'german_creditONEHOT_y', 'curet12000n2D_y', 'home', 'triangle1_y',
       'adult_y', 'ficoONEHOT', 'german_credit_y', '2d4cno9_y',
       '2d20cno0_y', 'fico', 'vehicle_y', 'zelnik6_y', 'titanic_y',
       'curet02000n2D_y', 'bank'], dtype=object)

In [71]:
DS1 = ['datasets/syntetic/2d-3c-no123_y.zip', 'datasets/syntetic/s-set1_y.zip', 'datasets/syntetic/2d-20c-no0_y.zip', 'datasets/syntetic/triangle2_y.zip',
                  'datasets/syntetic/triangle1_y.zip', 'datasets/syntetic/cure-t0-2000n-2D_y.zip', 'datasets/syntetic/longsquare_y.zip', 'datasets/syntetic/aggregation_y.zip',
                  'datasets/syntetic/2d-4c-no9_y.zip', 'datasets/syntetic/2d-d31_y.zip', 'datasets/syntetic/cure-t2-4k_y.zip', 'datasets/syntetic/zelnik6_y.zip',
                  'datasets/syntetic/2d-10c_y.zip', 'datasets/syntetic/2d-4c_y.zip', 'datasets/syntetic/zelnik5_y.zip', 'datasets/syntetic/cure-t1-2000n-2D_y.zip',
                  'datasets/syntetic/s-set2_y.zip', 'datasets/syntetic/tetra_y.zip']

DS1 = list(map(lambda x: x.split("/")[-1][:-4].replace("-", ""), DS1))

DS1

['2d3cno123_y',
 'sset1_y',
 '2d20cno0_y',
 'triangle2_y',
 'triangle1_y',
 'curet02000n2D_y',
 'longsquare_y',
 'aggregation_y',
 '2d4cno9_y',
 '2dd31_y',
 'curet24k_y',
 'zelnik6_y',
 '2d10c_y',
 '2d4c_y',
 'zelnik5_y',
 'curet12000n2D_y',
 'sset2_y',
 'tetra_y']

In [72]:
DS2 = [x for x in df.dataset.unique() if "ONEHOT" in x] + ["compasscorestwoyears_y"] + ["iris_y", "wdbc_y"]

DS2

['churnONEHOT_y',
 'compasscorestwoyearsONEHOT_y',
 'adultONEHOT_y',
 'german_creditONEHOT_y',
 'ficoONEHOT',
 'compasscorestwoyears_y',
 'iris_y',
 'wdbc_y']

In [73]:
DS3 = [x for x in df.dataset.unique() if x not in DS1+DS2[:-2]]

DS3

['wine',
 'churn_y',
 'diabetes_y',
 'iris_y',
 'wdbc_y',
 'ionosphere_y',
 'home',
 'adult_y',
 'german_credit_y',
 'fico',
 'vehicle_y',
 'titanic_y',
 'bank']

In [74]:
DS4 = [x for x in df.dataset.unique() if x not in DS1+DS2[:-3]]

DS4

['wine',
 'churn_y',
 'compasscorestwoyears_y',
 'diabetes_y',
 'iris_y',
 'wdbc_y',
 'ionosphere_y',
 'home',
 'adult_y',
 'german_credit_y',
 'fico',
 'vehicle_y',
 'titanic_y',
 'bank']

In [75]:
DS_name = "DS3"

In [76]:
if DS_name == "DS1":
    DS = DS1
    max_nbr_values_list = [100, 1000, np.inf]
    max_nbr_values_cat_list = [20]
if DS_name == "DS2":
    DS = DS2
    max_nbr_values_list = [100, 1000, np.inf]
    max_nbr_values_cat_list = [20, 100]
if DS_name == "DS3":
    DS = DS3
    max_nbr_values_list = [20]
    max_nbr_values_cat_list = [20]
if DS_name == "DS4":
    DS = DS4
    max_nbr_values_list = [20, 100, 1000, np.inf]
    max_nbr_values_cat_list = [20, 100]

In [77]:
df[df.alg == "CLTree"].dataset.unique()

array(['diabetes_y', '2d10c_y', 'sset2_y', 'tetra_y', '2dd31_y',
       '2d4cno9_y', 'sset1_y', 'compasscorestwoyears_y', 'home',
       'aggregation_y', '2d4c_y', 'triangle1_y', 'ionosphere_y',
       'curet24k_y', 'zelnik5_y', 'german_credit_y', 'vehicle_y',
       'titanic_y', 'curet12000n2D_y', '2d3cno123_y', 'longsquare_y',
       'zelnik6_y', '2d20cno0_y', 'curet02000n2D_y', 'triangle2_y'],
      dtype=object)

In [78]:
df = df[df.dataset.isin(DS)].copy(deep=True)

df

,n_clusters,max_leaves,base_tree,max_nbr_values,max_nbr_values_cat,time,silhouette,calinski_harabasz,davies_bouldin,r_score,...,metric_cat,max_eps,n_components,oblique_splits,max_oblique_features,min_nr_instances,min_y,min_rd,threshold,branching_factor
0,NaN,NaN,NaN,20.0,20.0,0.935042,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,20.0,20.0,0.930818,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,20.0,20.0,0.932790,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20.0,20.0,0.988993,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,20.0,20.0,0.969439,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10.0,NaN,NaN,1000.0,100.0,0.107084,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10.0,NaN,NaN,1000.0,20.0,0.101034,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,10.0,NaN,NaN,20.0,20.0,0.134736,0.2740,172.3922,1.0655,0.5324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,10.0,NaN,NaN,inf,100.0,0.097218,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df.alg.unique()

array(['VarianceParTree', 'skl_DBSCAN', 'pyc_xmeans', 'skl_kmeans',
       'ImpurityParTree2', 'kmeans+dt', 'skl_bis_kmeans', 'ShallowTree',
       'skl_agglomerativeClust', 'CLTree', 'kmodes', 'CenterParTree2',
       'skl_OPTICS', 'skl_birch', 'PrincipalParTree', 'ExKMC'],
      dtype=object)

In [80]:
df = df[(df.alg == "CLTree")|(df.max_nbr_values.isin(max_nbr_values_list) & df.max_nbr_values_cat.isin(max_nbr_values_cat_list))].copy(deep=True)

df

,n_clusters,max_leaves,base_tree,max_nbr_values,max_nbr_values_cat,time,silhouette,calinski_harabasz,davies_bouldin,r_score,...,metric_cat,max_eps,n_components,oblique_splits,max_oblique_features,min_nr_instances,min_y,min_rd,threshold,branching_factor
0,NaN,NaN,NaN,20.0,20.0,0.935042,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,20.0,20.0,0.930818,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,20.0,20.0,0.932790,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20.0,20.0,0.988993,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,20.0,20.0,0.969439,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,6.0,NaN,NaN,20.0,20.0,6.247307,0.1537,1128.0498,1.8523,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,6.0,NaN,NaN,20.0,20.0,6.154932,0.1420,1085.7561,1.8761,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,NaN,NaN,20.0,20.0,0.083353,0.2740,172.3922,1.0655,0.5324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10.0,NaN,NaN,20.0,20.0,0.082231,0.2740,172.3922,1.0655,0.5324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df.alg.unique()

array(['VarianceParTree', 'skl_DBSCAN', 'pyc_xmeans', 'ImpurityParTree2',
       'skl_kmeans', 'kmeans+dt', 'skl_bis_kmeans',
       'skl_agglomerativeClust', 'CLTree', 'kmodes', 'CenterParTree2',
       'skl_OPTICS', 'skl_birch', 'PrincipalParTree'], dtype=object)

In [82]:
#order = "silhouette"
order = "norm_mutual_info_score"

metriche = dict([#Ascending?
    ('silhouette', False), #max
    ('time', True), #min
    ('calinski_harabasz', False),
    ('davies_bouldin', True),
    ('r_score', False),
    ('adj_rand', False),
    ('mut_info_score', False),
    ('adj_mutual_info_score', False),
    ('norm_mutual_info_score', False),
    ('homog_score', True),
    ('complete_score', False),
    ('v_msr_score', False),
    ('fwlks_mallows_score', False)
])

In [83]:
df_subset = df.sort_values(['dataset', 'alg', order, 'time'], ascending=[False, False, metriche[order], True]).groupby(['dataset', 'alg']).nth(0).reset_index()

df_subset[res_values_sup+["time"]]

,r_score,adj_rand,mut_info_score,adj_mutual_info_score,norm_mutual_info_score,homog_score,complete_score,v_msr_score,fwlks_mallows_score,time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.039842
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.959601
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.924591
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400090
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.206884
...,...,...,...,...,...,...,...,...,...,...
161,0.3987,-0.0082,0.0423,0.0305,0.0307,0.0767,0.0192,0.0307,0.2912,1.406859
162,0.4640,0.0086,0.0205,0.0201,0.0202,0.0371,0.0139,0.0202,0.4752,10.019329
163,0.5317,0.1526,0.1249,0.1363,0.1363,0.2262,0.0976,0.1363,0.5351,2.836186
164,0.5157,0.0973,0.0845,0.0996,0.0996,0.1530,0.0739,0.0996,0.5350,3.665071


In [84]:
df_subset.to_csv(f"{DS_name} order by {order}.csv")

In [85]:
#df_subset["rank_per_dataset"] = df_subset.groupby("dataset")[order].rank(ascending=metriche[order]).astype(int)

#df_subset.sort_values(["dataset", "rank_per_dataset"])[["rank_per_dataset", order, "alg", "dataset"]]

In [86]:
df_agg = df_subset.groupby("alg")[res_values_all].agg(["mean", "std", "min", "max"])

df_agg.head()

time                                 silhouette  \
                       mean        std       min        max       mean   
alg                                                                      
CLTree            33.329146  33.756669  6.983112  81.199169  -0.124550   
CenterParTree2     2.470815   4.458641  0.180959  16.350173   0.058517   
ImpurityParTree2   1.370930   1.464575  0.131042   5.089561   0.030023   
PrincipalParTree   1.134377   1.480306  0.082807   5.440709   0.138138   
VarianceParTree    2.187941   3.281275  0.116270  10.019329   0.166931   

                                           calinski_harabasz                \
                       std     min     max              mean           std   
alg                                                                          
CLTree            0.100528 -0.2993  0.0020          1.400967      0.493956   
CenterParTree2    0.214642 -0.1947  0.6575        994.339000   2272.045289   
ImpurityParTree2  0.251135 -0.4183  0.6613        827.995654   2151.047795   
PrincipalParTree  0.201978 -0.1155  0.6613       1520.926877   3300.019518   
VarianceParTree   0.359205 -0.1573  0.9253       5259.839592  14875.712691   

                  ... complete_score         v_msr_score                    \
                  ...            min     max        mean       std     min   
alg               ...                                                        
CLTree            ...         0.0337  0.3518     0.13425  0.083163  0.0544   
CenterParTree2    ...         0.0097  0.4020     0.18559  0.149642  0.0114   
ImpurityParTree2  ...         0.0093  0.4615     0.17143  0.178829  0.0121   
PrincipalParTree  ...         0.0190  0.7388     0.26346  0.250225  0.0293   
VarianceParTree   ...         0.0057  0.5257     0.17128  0.191316  0.0074   

                         fwlks_mallows_score                            
                     max                mean       std     min     max  
alg                                                                     
CLTree            0.2574            0.514067  0.195204  0.2506  0.7700  
CenterParTree2    0.4699            0.502090  0.162642  0.2613  0.6860  
ImpurityParTree2  0.5177            0.558640  0.152643  0.3218  0.7852  
PrincipalParTree  0.7541            0.557890  0.217072  0.3279  0.8554  
VarianceParTree   0.5829            0.540670  0.160741  0.3244  0.7852  

[5 rows x 52 columns]

In [87]:
def getWin(metrica):
    ascending = metriche[metrica]
    df_tmp = df_subset[["alg", "dataset", metrica]].copy()
    df_tmp["rank"] = df_subset.groupby("dataset")[metrica].rank(ascending=ascending, method="min", na_option="top" if ascending else "bottom").astype(int)
    return df_tmp.groupby("alg")[["rank"]].agg(c=("rank", len), win=("rank", lambda x: len(x[x == 1])), rank_mean=("rank", np.mean))

In [88]:
df_agg = df_agg[df_agg.index != "pyc_agglomerativeClust"].copy(deep=True)

for metrica in res_values_all:
    df_tmp = getWin(metrica)
    df_agg[(metrica, "win")] = df_tmp.win
    df_agg[(metrica, "count")] = df_tmp.c
    df_agg[(metrica, "rank_mean")] = df_tmp.rank_mean

df_agg[df_agg.columns.sort_values()]

adj_mutual_info_score                            \
                                       count     max      mean     min   
alg                                                                      
CLTree                                     6  0.2407  0.126483  0.0455   
CenterParTree2                            12  0.4552  0.181520  0.0111   
ImpurityParTree2                          13  0.4994  0.166520  0.0071   
PrincipalParTree                          13  0.7491  0.259990  0.0251   
VarianceParTree                           13  0.5700  0.165110  0.0045   
kmeans+dt                                 13  0.7316  0.192000  0.0012   
kmodes                                    12  0.3208  0.130050  0.0016   
pyc_xmeans                                12  0.7316  0.223010  0.0019   
skl_DBSCAN                                12  0.6930  0.219560  0.0044   
skl_OPTICS                                12  0.7316  0.150930  0.0097   
skl_agglomerativeClust                    12  0.7316  0.244350  0.0199   
skl_birch                                 12  0.6900  0.206750  0.0026   
skl_bis_kmeans                            12  0.7316  0.214380  0.0019   
skl_kmeans                                12  0.7316  0.206240  0.0018   

                                               adj_rand                    \
                       rank_mean       std win    count     max      mean   
alg                                                                         
CLTree                  8.833333  0.080814   1        6  0.1976  0.090917   
CenterParTree2          6.166667  0.147214   4       12  0.3373  0.141080   
ImpurityParTree2        7.153846  0.176180   3       13  0.5335  0.146370   
PrincipalParTree        2.230769  0.250001   6       13  0.7234  0.276390   
VarianceParTree         6.846154  0.189922   3       13  0.5335  0.134660   
kmeans+dt               7.076923  0.238528   3       13  0.5754  0.160500   
kmodes                  8.666667  0.106361   2       12  0.2530  0.090550   
pyc_xmeans              3.833333  0.238804   3       12  0.6536  0.195380   
skl_DBSCAN              6.333333  0.255227   2       12  0.6548  0.198020   
skl_OPTICS              7.250000  0.218106   2       12  0.5681  0.071400   
skl_agglomerativeClust  3.500000  0.260236   5       12  0.7299  0.224060   
skl_birch               6.333333  0.229838   2       12  0.5865  0.191060   
skl_bis_kmeans          5.750000  0.244674   2       12  0.6536  0.203310   
skl_kmeans              6.166667  0.244034   2       12  0.6536  0.182980   

                        ...       time                v_msr_score          \
                        ...  rank_mean        std win       count     max   
alg                     ...                                                 
CLTree                  ...  14.000000  33.756669   0           6  0.2574   
CenterParTree2          ...  10.166667   4.458641   0          12  0.4699   
ImpurityParTree2        ...   8.384615   1.464575   0          13  0.5177   
PrincipalParTree        ...   6.923077   1.480306   0          13  0.7541   
VarianceParTree         ...   8.076923   3.281275   0          13  0.5829   
kmeans+dt               ...   5.692308   0.442935   1          13  0.7337   
kmodes                  ...   9.833333   1.842679   0          12  0.3551   
pyc_xmeans              ...   2.250000   0.284549   3          12  0.7337   
skl_DBSCAN              ...   2.000000   0.796760   4          12  0.7006   
skl_OPTICS              ...  12.833333  59.917891   0          12  0.7337   
skl_agglomerativeClust  ...   3.083333  13.510029   5          12  0.7337   
skl_birch               ...   5.583333   4.436615   0          12  0.6925   
skl_bis_kmeans          ...   8.083333   3.532766   0          12  0.7337   
skl_kmeans              ...   6.500000   0.451834   0          12  0.7337   

                                                                 
                           mean     min rank_mean       std win  
alg         

In [89]:
lista = [ 
    'CenterParTree2', 
    'ImpurityParTree2', 
    'PrincipalParTree',
    'VarianceParTree', 
    'CLTree',
    'kmeans+dt2', 
    
    'skl_kmeans',
    'skl_bis_kmeans',
    'pyc_xmeans',
    'kmodes', 
    'skl_birch', 
    'skl_agglomerativeClust', 
    'skl_DBSCAN',
    'skl_OPTICS', 
    'skl_agglomerativeClust',     
]
df_agg["indice"] = list(map(lambda x: lista.index(x), df_agg.index.tolist()))
df_agg[df_agg.columns.sort_values()].sort_values(by="indice").drop(columns="indice")\
        .to_excel(f"{DS_name} order by {order}.xlsx")

ValueError: 'kmeans+dt' is not in list

In [ ]:
df.alg.unique()